In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [14]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


# No Line Search

In [15]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=0.01, line_search_method="const")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.07094807922840118
epoch:  1, loss: 0.07073919475078583
epoch:  2, loss: 0.06263881921768188
epoch:  3, loss: 0.05328408256173134
epoch:  4, loss: 0.04686925932765007
epoch:  5, loss: 0.042571764439344406
epoch:  6, loss: 0.0376533605158329
epoch:  7, loss: 0.033686183393001556
epoch:  8, loss: 0.029170960187911987
epoch:  9, loss: 0.025123022496700287
epoch:  10, loss: 0.022134754806756973
epoch:  11, loss: 0.01996367797255516
epoch:  12, loss: 0.018057124689221382
epoch:  13, loss: 0.017203230410814285
epoch:  14, loss: 0.015816818922758102
epoch:  15, loss: 0.014692587777972221
epoch:  16, loss: 0.013666029088199139
epoch:  17, loss: 0.01296538207679987
epoch:  18, loss: 0.012154581025242805
epoch:  19, loss: 0.011442584916949272
epoch:  20, loss: 0.010801419615745544
epoch:  21, loss: 0.01023920252919197
epoch:  22, loss: 0.009737319312989712
epoch:  23, loss: 0.00929289311170578
epoch:  24, loss: 0.008897455409169197
epoch:  25, loss: 0.008433111011981964
epoch: 

In [16]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.8602228165972577
Test metrics:  R2 = 0.7169409821393211


# Backtracking Line search

In [18]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.20084208250045776
epoch:  1, loss: 0.16193899512290955
epoch:  2, loss: 0.13151630759239197
epoch:  3, loss: 0.11025375127792358
epoch:  4, loss: 0.09014838933944702
epoch:  5, loss: 0.07248212397098541
epoch:  6, loss: 0.06264548003673553
epoch:  7, loss: 0.0562414713203907
epoch:  8, loss: 0.04392985254526138
epoch:  9, loss: 0.03789079561829567
epoch:  10, loss: 0.03386625647544861
epoch:  11, loss: 0.017122888937592506
epoch:  12, loss: 0.011662391945719719
epoch:  13, loss: 0.011443219147622585
epoch:  14, loss: 0.011242805048823357
epoch:  15, loss: 0.011061047203838825
epoch:  16, loss: 0.010904776863753796
epoch:  17, loss: 0.00961024034768343
epoch:  18, loss: 0.008927300572395325
epoch:  19, loss: 0.008174488320946693
epoch:  20, loss: 0.007604735903441906
epoch:  21, loss: 0.007157898973673582
epoch:  22, loss: 0.006742303259670734
epoch:  23, loss: 0.006435425020754337
epoch:  24, loss: 0.0061398763209581375
epoch:  25, loss: 0.0059205880388617516
epoch: 

In [19]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.8838890850751601
Test metrics:  R2 = 0.538304836613132


# Bisection Line search

In [22]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="goldstein")
opt = torch_numopt.NewtonRaphson(params=model.parameters(), model=model, lr=1, line_search_method="bisect")

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.40123647451400757
epoch:  1, loss: 0.35990622639656067
epoch:  2, loss: 0.05912674590945244
epoch:  3, loss: 0.028653740882873535
epoch:  4, loss: 0.016193212941288948
epoch:  5, loss: 0.012228498235344887
epoch:  6, loss: 0.010191763751208782
epoch:  7, loss: 0.009348577819764614
epoch:  8, loss: 0.00750616192817688
epoch:  9, loss: 0.006780543830245733
epoch:  10, loss: 0.005453215911984444
epoch:  11, loss: 0.004122577141970396
epoch:  12, loss: 0.0035072064492851496
epoch:  13, loss: 0.002601299434900284
epoch:  14, loss: 0.002207098761573434
epoch:  15, loss: 0.001885295263491571
epoch:  16, loss: 0.0017658089054748416
epoch:  17, loss: 0.001580837881192565
epoch:  18, loss: 0.0014616461703553796
epoch:  19, loss: 0.0013719871640205383
epoch:  20, loss: 0.0012632940197363496
epoch:  21, loss: 0.00125051976647228
epoch:  22, loss: 0.0011980141280218959
epoch:  23, loss: 0.001125263050198555
epoch:  24, loss: 0.001077914610505104
epoch:  25, loss: 0.00103347597178

In [23]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.990648005655829
Test metrics:  R2 = 0.8949833603702978
